In [61]:
import pandas as pd
import numpy as np

In [62]:
df=pd.read_csv('../artifacts/glassdoor_jobs.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Manager Data Scientist,NaN,We are seeking a highly skilled and experience...,NaN,Optum,Bengaluru,NaN,NaN,NaN,NaN,NaN,NaN
1,"Lead Data Scientist, India - BCG X",₹6L – ₹9L/yr (Glassdoor est.),Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
2,Global Data Scientist Senior Manager,NaN,WHAT YOU'LL DO\n\nAs part of the Digital produ...,4.2,Boston Consulting Group,Delhi,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD)
3,Sr AI/ML Engineer,NaN,OptumAI is looking for a Senior Machine Learni...,NaN,Optum,Haryāna,NaN,NaN,NaN,NaN,NaN,NaN
4,Principal AI or ML Engineer,NaN,"As the Principal AI/ML Engineer, you will be r...",NaN,Optum,India,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Size                  object
Founded               object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [64]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'
    
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'

In [65]:
df['job_simp'] = df['Job Title'].apply(title_simplifier)
df['seniority'] = df['Job Title'].apply(seniority)

In [66]:
df['Salary Estimate'].isna().sum()

203

In [67]:
df = df.dropna(subset=['Salary Estimate'])

In [68]:
# Remove currency symbols and extraneous text
df['Salary Estimate'] = df['Salary Estimate'].str.replace('₹', '').str.replace('L', '*100000').str.replace('T', '*1000')
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'\(Glassdoor est.\)', '', regex=True)
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'\(Employer est.\)', '', regex=True)
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'/yr', '', regex=True)
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'/mo', '*12', regex=True)
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'/hr', '*2080', regex=True)  # Assuming 40 hours per week and 52 weeks per year

In [69]:
df['avg_salary'] = df['Salary Estimate'].apply(lambda x: eval(x.split('–')[0].strip())+eval(x.split('–')[1].strip()) if '–' in str(x) else eval(x))

In [70]:
df['avg_salary'].value_counts()

avg_salary
1900000.0    101
1800000.0     68
400000.0      67
800000.0      67
1300000.0     66
1500000.0     34
6100000.0     34
252000.0      34
500000.0      34
2400000.0     33
2080000.0     33
300000.0      33
120000.0      33
1400000.0     32
900000.0      32
2600000.0     32
125000.0      32
1100000.0     32
Name: count, dtype: int64

In [71]:
#parsing 'Job Description'
skills = {
    'python': 'python_yn',
    'sql': 'sql_yn',
    'excel': 'excel_yn',
    'spark': 'spark_yn',
    'aws': 'aws_yn'
}
# Loop through the skills dictionary to create and populate the columns
for skill, column in skills.items():
    df[column] = df['Job Description'].str.contains(skill, case=False, na=False).astype(int)

In [72]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_simp,seniority,avg_salary,python_yn,sql_yn,excel_yn,spark_yn,aws_yn
1,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000+ Employees,1963,Company - Private,Business consulting,Management and consulting,$5 to $10 billion (USD),data scientist,senior,1500000.0,0,0,0,1,0
5,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000+ Employees,1996,Company - Public,Internet & Web Services,Information Technology,$5 to $10 billion (USD),mle,senior,1800000.0,0,0,0,1,1
6,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000+ Employees,1690,Company - Public,Banking & Lending,Finance,$10+ billion (USD),data scientist,na,1900000.0,0,0,0,0,0
7,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,NaN,NaN,data scientist,na,6100000.0,1,0,0,0,0
8,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,NaN,NaN,NaN,NaN,NaN,NaN,na,na,252000.0,1,0,1,0,0


In [73]:
df['Size']=df['Size'].str.replace('Employees',' ')
df['Size']=df['Size'].str.replace('+',' ')
df['Size']=df['Size'].str.replace('to','-')

In [74]:
# Extract the upper limit of the size range
df['Size_Upper'] = df['Size'].apply(lambda x: x.split('-')[1].strip() if '-' in str(x) else x)

In [75]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Revenue,job_simp,seniority,avg_salary,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper
1,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963,Company - Private,Business consulting,...,$5 to $10 billion (USD),data scientist,senior,1500000.0,0,0,0,1,0,10000
5,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000,1996,Company - Public,Internet & Web Services,...,$5 to $10 billion (USD),mle,senior,1800000.0,0,0,0,1,1,10000
6,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000,1690,Company - Public,Banking & Lending,...,$10+ billion (USD),data scientist,na,1900000.0,0,0,0,0,0,10000
7,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,...,NaN,data scientist,na,6100000.0,1,0,0,0,0,NaN
8,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,NaN,NaN,NaN,NaN,...,NaN,na,na,252000.0,1,0,1,0,0,NaN


In [76]:
df['Founded'].value_counts()

Founded
--      99
2007    68
1963    67
1996    34
1690    34
1860    34
1967    34
2019    33
2006    33
1851    32
1889    32
1998    32
Name: count, dtype: int64

In [77]:
# Convert the 'Founded' column to numeric, forcing errors(i.e '--' values) to NaN
df['Founded'] = pd.to_numeric(df['Founded'], errors='coerce')

In [78]:
df['Age'] = 2024 - df['Founded'].dropna().astype(int) # considering nan values

In [79]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,job_simp,seniority,avg_salary,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper,Age
1,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963.0,Company - Private,Business consulting,...,data scientist,senior,1500000.0,0,0,0,1,0,10000,61.0
5,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000,1996.0,Company - Public,Internet & Web Services,...,mle,senior,1800000.0,0,0,0,1,1,10000,28.0
6,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000,1690.0,Company - Public,Banking & Lending,...,data scientist,na,1900000.0,0,0,0,0,0,10000,334.0
7,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,...,data scientist,na,6100000.0,1,0,0,0,0,NaN,NaN
8,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,NaN,NaN,NaN,NaN,...,na,na,252000.0,1,0,1,0,0,NaN,NaN


In [80]:
df['Type of ownership'].value_counts()

Type of ownership
Company - Private                 267
Company - Public                  168
Unknown                            33
Subsidiary or Business Segment     32
Private Practice / Firm            32
Name: count, dtype: int64

In [81]:
df['Industry'].value_counts()

Industry
Information Technology Support Services    134
Business consulting                         67
--                                          66
Internet & Web Services                     34
Banking & Lending                           34
Research and development                    34
Electronics Manufacturing                   34
Computer Hardware Development               33
Crop Production                             32
Machinery Manufacturing                     32
HR Consulting                               32
Name: count, dtype: int64

In [82]:
df['Industry'] = df['Industry'].apply(lambda x: 'unknown' if x == '--' else x)

In [83]:
df['Sector'].value_counts()

Sector
Information Technology          201
Management and consulting       101
Manufacturing                    66
--                               66
Finance                          34
Agriculture                      32
Human resources and staffing     32
Name: count, dtype: int64

In [84]:
df['Sector'] = df['Sector'].apply(lambda x: 'unknown' if x == '--' else x)

In [85]:
df['Revenue'].value_counts()

Revenue
$10+ billion (USD)          166
Unknown / Non-Applicable    164
$5 to $10 billion (USD)     101
$5 to $25 million (USD)     101
Name: count, dtype: int64

In [86]:
df['Revenue'] = df['Revenue'].str.strip() \
    .str.replace(r'\$', '', regex=True) \
    .str.replace(r'to', '-', regex=False) \
    .str.replace(r'\+', '', regex=True) \
    .str.replace(r'billion', '000', regex=False) \
    .str.replace(r'million', '', regex=False) \
    .str.replace(r'Unknown / Non-Applicable', 'unknown', regex=False) \
    .str.replace(r'USD', '', regex=False) \
    .str.replace(r'\(', '', regex=True) \
    .str.replace(r'\)', '', regex=True)

    

In [87]:
df['Revenue_Upper'] = df['Revenue'].apply(lambda x: x.split('-')[1].strip() if '-' in str(x) else x)

In [88]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,seniority,avg_salary,python_yn,sql_yn,excel_yn,spark_yn,aws_yn,Size_Upper,Age,Revenue_Upper
1,"Lead Data Scientist, India - BCG X",6*100000 – 9*100000,Locations: Mumbai | Gurgaon\nWho We Are\nBosto...,4.2,Boston Consulting Group,Gurgaon,10000,1963.0,Company - Private,Business consulting,...,senior,1500000.0,0,0,0,1,0,10000,61.0,10 000
5,Senior Machine Learning Engineer,8*100000 – 10*100000,If you need assistance during the recruiting p...,3.8,Expedia Partner Solutions,Gurgaon,10000,1996.0,Company - Public,Internet & Web Services,...,senior,1800000.0,0,0,0,1,1,10000,28.0,10 000
6,Data Scientist,9*100000 – 10*100000,.\nPurpose of the role\nTo use innovative data...,3.9,Barclays,Chennai,10000,1690.0,Company - Public,Banking & Lending,...,na,1900000.0,0,0,0,0,0,10000,334.0,10 000
7,Python Data Scientist/Analyst,25*100000 – 36*100000,"Bachelor’s/Master’s degree in Engineering, Com...",NaN,Excellent Opportunity,Remote,NaN,NaN,NaN,NaN,...,na,6100000.0,1,0,0,0,0,NaN,NaN,NaN
8,Python & Data Science Trainer,12*1000 – 20*1000*12,Responsible for delivering workshops & trainin...,NaN,IPCS Global solutions pvt ltd,India,NaN,NaN,NaN,NaN,...,na,252000.0,1,0,1,0,0,NaN,NaN,NaN


In [89]:
df['Revenue_Upper'].value_counts()

Revenue_Upper
10 000     166
unknown    164
10 000     101
25         101
Name: count, dtype: int64